Collecting Data with User Name Prompt

In [6]:
import cv2
import os
import time

# Prompt user for their name
user_name = input("Please enter your name: ").strip()

# Main directory to save the collected images
main_data_dir = 'Dataset'

# Subdirectory for the user
user_dir = os.path.join(main_data_dir, user_name)

if not os.path.exists(user_dir):
    os.makedirs(user_dir)
    print(f"Directory created for {user_name}. Proceeding to collect data...")
else:
    print(f"Directory for {user_name} already exists. Proceeding to collect data...")

# Initialize the webcam
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

count = 0
while count < 100:  # Capture 100 images
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        cv2.imwrite(os.path.join(user_dir, f'{user_name}_{count}.jpg'), face)
        count += 1
        time.sleep(0.2)  # Add a 0.5 seconds delay
    
    cv2.imshow('Collecting Your Face Data', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

Directory created for G. Proceeding to collect data...


Training the Model

In [7]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Load your dataset
def load_data(data_dir):
    images = []
    labels = []
    label_dict = {}
    current_label = 0
    
    for person_name in os.listdir(data_dir):
        person_dir = os.path.join(data_dir, person_name)
        if not os.path.isdir(person_dir):
            continue
        
        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            image = cv2.resize(image, (128, 128))
            images.append(image)
            if person_name not in label_dict:
                label_dict[person_name] = current_label
                current_label += 1
            labels.append(label_dict[person_name])
    
    images = np.array(images).reshape(-1, 128, 128, 1)
    labels = np.array(labels)
    return images, labels, label_dict

data_dir = 'Dataset'
images, labels, label_dict = load_data(data_dir)

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

# Build the model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(label_dict), activation='softmax'))  # Use softmax for multi-class classification

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

# Save the model and label dictionary
model.save('face_recognition_model.h5')
np.save('label_dict.npy', label_dict)


Epoch 1/20
4/4 [==============================] - 1s 267ms/step - loss: 10.6433 - accuracy: 0.6719 - val_loss: 0.1686 - val_accuracy: 0.9688
Epoch 2/20
4/4 [==============================] - 1s 226ms/step - loss: 0.4971 - accuracy: 0.9453 - val_loss: 0.2903 - val_accuracy: 0.9688
Epoch 3/20
4/4 [==============================] - 1s 211ms/step - loss: 0.2806 - accuracy: 0.9688 - val_loss: 2.5789e-04 - val_accuracy: 1.0000
Epoch 4/20
4/4 [==============================] - 1s 207ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 5/20
4/4 [==============================] - 1s 224ms/step - loss: 0.0173 - accuracy: 0.9922 - val_loss: 1.3075 - val_accuracy: 0.9375
Epoch 6/20
4/4 [==============================] - 1s 208ms/step - loss: 0.2767 - accuracy: 0.9766 - val_loss: 0.3191 - val_accuracy: 0.9688
Epoch 7/20
4/4 [==============================] - 1s 220ms/step - loss: 9.1269e-08 - accuracy: 1.0000 - val_loss: 4.4703e-07 - val_accuracy: 1.0000

Real-time Face Detection and Recognition

In [8]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load the pre-trained model and label dictionary
model = load_model('face_recognition_model.h5')
label_dict = np.load('label_dict.npy', allow_pickle=True).item()

def preprocess_image(image):
    image = cv2.resize(image, (128, 128))
    image = image.reshape(1, 128, 128, 1)
    image = image / 255.0
    return image

def predict_face(image, model, label_dict):
    processed_image = preprocess_image(image)
    prediction = model.predict(processed_image)
    label = np.argmax(prediction)
    confidence = np.max(prediction)
    if confidence > 0.5:  # Confidence threshold
        return list(label_dict.keys())[list(label_dict.values()).index(label)], confidence
    else:
        return "Unknown", confidence

# Initialize the webcam
cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = gray[y:y+h, x:x+w]
        label, confidence = predict_face(face, model, label_dict)
        
        if label != "Unknown":
            color = (0, 255, 0)
        else:
            color = (0, 0, 255)
        
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
        cv2.putText(frame, f'{label} ({confidence*100:.2f}%)', (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    cv2.imshow('Face Recognition', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 20ms/step
